In [27]:
#####
# This code block contains a naive precomputation method:
# It inputs a generator of GF(2^n) and returns a list of
# conjugates:
#    conjugates <-- [ (a^i)^{2^j}, 0 <= i,j <= n-1 ]
# and also a list of the traces of each power:
#    traces     <-- [ Tr(a^i), 0 <= i <= n-1 ]

def precompute(n,a):
    conjugates = [ [(a**i)**(2**j) for j in range(n)] for i in range(n) ]
    traces = [cong[0].trace() for cong in conjugates]
    return conjugates, traces

In [13]:
######
# This code block contains conjugate update methods.
# There is very little reason to use anything but update_conjugates_graycode 
# update_conjugates_naive is implemented primarily for testing purposes
######

# This takes in an element and outputs a list of conjugates by repeated squaring
def update_conjugates_naive(conjugates, dummy_1, dummy_2): #Not the most graceful way of vargs
    n = len(conjugates)
    conj = []
    conj.append(conjugates[0])
    for i in range(1,n):
        conj.append(conj[i-1]**2)
    return conj

def update_conjugates_graycode(current_conjugates, basic_conjugates, i):
    n = len(current_conjugates)
    for j in range(n): #Use the Gray code for efficient updating
        current_conjugates[j] += basic_conjugates[i][j]
    return current_conjugates

In [14]:
###########
# These are various canonical element methods. 
# Each method takes in a list of elements and returns true if
# the first is a canonical element; e.g., the elements should
# all be representable as bit strings, so these will return true
# if the first element is lex_first.

# The efficiency of these methods is more crucial than you might
# expect. On a former version, in C, I found we were spending 33%
# of our time in a lex-first checker.
###########
    
# This method checks the bitstring bit-wise.
def is_lex_first_vector_compare(k):
    n = len(k)
    current = k[0]._vector_()
    for i in range(1,n):
        next = k[i]._vector_()
        for j in range(n):
            if next[j] == 1 and current[j] == 0:
                return False
            elif next[j] == 0 and current[j] == 1:
                break
    return True

# This method uses the built-in ``integer_representation()'' function
# and returns true if the first element is the lex_minimal.
def is_lex_first_int_rep(k):
    n = len(k)
    current = k[0].integer_representation()
    for i in range(1,n):
    #for i in my_order:
        next = k[i].integer_representation()
        if next >= current:
            return False
    return True

# This method again uses ``integer_representation()'' but sorts the
# resulting integer array. Parallelizable.
def is_lex_first_int_rep_sort(k):
    #n = len(k)
    #int_reps = [k[i].integer_representation() for i in xrange(n)]
    int_reps = [ kk.integer_representation() for kk in k ]
    int_reps.sort()
    if int_reps[0] == k[0].integer_representation():
        return True
    else:
        return False

# This method directly sorts the bit strings and returns true if the
# first element is the lex_minimal element.
def is_lex_first_sort(k):
    current = k[0]
    k.sort()
    if current == k[0]:
        return True
    else:
        return False

In [15]:
############################
# These methods take a list of conjugates and count the complexity of the normal basis.
############################
    
def get_complexity_naive(k, dummy):
    #Naive means compute \alpha \alpha^{q^i} = \sum_{k=0}^{n-1} t_{ij} \alpha_k for all i and count.
    #We want the change of basis matrix P such that xP = b, where x is the coordinate vector in the
    #normal basis and b is the coordinate vector in the polynomial basis. Really, of course, we want
    #x = b inv(P)

    #Construct a matrix P
    n = len(k)
    P = matrix( [ k[i]._vector_() for i in range(n) ] )
    try: 
        P_inv = P.inverse()
    except:
        return n**2
    #Whole matrix computation
    cplex = 0
    B = matrix( [ (k[0] * k[i])._vector_() for i in range(n)] )
    T = B*P_inv
    cplex += len(T.nonzero_positions())
    return cplex

# This method computes the multiplication table row-by-row, 
# and early-abort if the complexity of the current element 
# exceeds a threshold. 
# Experimental
def get_complexity_early_abort(k, min_cplex):
    n = len(k)
    #Construct a matrix P
    P = matrix( [ k[i]._vector_() for i in range(n) ] )
    try:
        P_inv = P.inverse()
    except:
        return n**2
    #row-by-row computation
    cplex = 1
    for i in range(1,n):
        bi = k[0] * k[i] #compute \alpha \alpha^{q^i}
        row = bi._vector_() * P_inv
        cplex += row.hamming_weight()
        if cplex > min_cplex:
            return cplex
    return cplex

In [16]:
################################
# This section implements normality checkers: Given a set of conjugates,
# determine if they represent a normal basis.
################################

# This method inputs a list of conjugates and a list of factors of x^n-1 and returns true if none of the factors of x^n-1 divide the auxiliary polynomial.
def is_normal_factors(g, h, cyclotomic_factors):
    #n = len(k)
    #g = k[0]*x**(n-1)
    #for i in range(1,n):
    #    g += k[i]*x**(n-1-i)
    for i in range(1,len(cyclotomic_factors)):
        if cyclotomic_factors[i][0].divides(g):
            return False
    return True

# This is a dummy function when normality need not be checked
def no_normal_check(k, h, dummy):
    return True

# This method constructs the auxiliary polynomial as above, but directly 
# runs an extension field GCD
def is_normal_naive(g, h, dummy):
    #n = len(k)
    #g = Kx(reversed(k))
    #g = k[0]*x**(n-1)
    #for i in range(1,n):
    #    g += k[i]*x**(n-1-i)
    if GCD(g, h) == 1:
        return True
    else:
        return False

In [17]:
#This code block contains the main search object.
class Normal_Search:
    def __init__(self, n, cplex, lex, normal, precompute, update_conjugates):
        self.n = n #left n as a global. So it's bad form, sue me.
        self.precompute = precompute
        self.check_complexity = cplex
        self.is_lex_first = lex
        self.is_normal = normal
        self.update_conjugates = update_conjugates
        self.F = GF(2)
        K.<a> = GF(2**n)
        self.K = K
        self.a = a
        Kx.<x> = self.K[]
        self.x = x
        self.Kx = Kx
        self.ex_n_less_one = x**n-1
        
    def linearized_associate(self, f):
        flist = f.list()
        l = 0
        for i in range(len(flist)):
            if flist[i] == 1:
                l += self.x**(2**i)
        return l
        
    def run(self):

        #### Gray code magic; the index of the element to flip is the number of trailing zero bits of the integer i.
        GC_iter = (i.trailing_zero_bits() for i in srange(1, 2^self.n)) # Initialize the Gray code iterator.

        # Construct the basic_conjugates[i][j] = e_i^{2^j}, where e_i is the i-th standard basis element.
        basic_conjugates, traces = self.precompute(self.n, self.a)
        
        # These are precomputed factors of x^n-1 and used only in is_normal_factors. 
        # These lines could and should be moved
        cyclotomic_factors = list(self.ex_n_less_one.factor())
        lin_list = [ self.linearized_associate(l[0]) for l in cyclotomic_factors ]

        #Initialize zero-valued
        current = self.K.zero()
        conjugates = [self.K.zero()]*self.n
        complexities = [0] * (self.n**2+1) # +1 to fix a bug -- if the first element doesn't pass the complexity check, check_complexity returns n^2, which is out of the list index
        min_cplex = self.n**2
        min_elements = [self.K.zero()]
        my_trace = 0
        count = 0
        
        #Main outer loop
        for i in GC_iter:
            #Get the next element and all its conjugates
            conjugates = self.update_conjugates(conjugates, basic_conjugates, i) #Need to homogenize args
            
            # Adding in trace computation is obviously better, so we don't add this as an option
            # If you are REALLY interested, you can compare the effect by commenting out the next
            # 3 lines. 
            my_trace += traces[i]
            if my_trace == 0: 
                continue

            if self.is_lex_first(conjugates): #Check canonicity
                g = self.Kx(conjugates[::-1]) #Create a polynomial with coefficients are conjugates in reverse order
                if self.is_normal(g, self.ex_n_less_one, cyclotomic_factors): #Check normality
                    cplex = self.check_complexity(conjugates, min_cplex)      #Check complexity
                    if cplex <= min_cplex:
                            complexities[cplex] += 1 #Keep a histogram of complexities
                            if cplex < min_cplex:    #Allow multiple minimum complexity elements. Rare.
                                min_cplex = cplex
                                min_elements = [conjugates[0]]
                            else:
                                min_elements.append(conjugates[0])
                    else:
                            continue
        print "Minimum complexity found: %d"%min_cplex
        print "Minimal polynomial of minimal complexity element: %s"%min_elements[0].minimal_polynomial()
        ##Uncomment this to output histogram of complexities
        #seen_complexities = [(i, complexities[i]) for i in range(len(complexities)) if complexities[i] != 0]

In [ ]:
#Below is the run method. Assign the desired parameters:
# n                 : degree of extension
# complexity_check  : Method of checking complexity
# lex_first_check   : Method of checking canonical basis element
# normality_check   : Method of checking the canonical element is normal
# run_precompute    : Method determining which values are precomputed
# update_conjugates : Method of determining next element and its conjugates

#Settings for Algorithm 2
complexity_check = get_complexity_naive
lex_first_check  = is_lex_first_int_rep
normality_check  = no_normal_check
run_precompute   = precompute
update_conjugates= update_conjugates_graycode

#Settings for normality checking
is_normal = [is_normal_naive, no_normal_check]

#Options for complexity checking
get_complexity = [get_complexity_naive, get_complexity_early_abort]

#Options for canonicity checking
lex_first = [is_lex_first_int_rep, is_lex_first_vector_compare]

#Options for precomputation
precomp = [precompute]


#Here you can loop over a set of n, or sets of functions for comparative timings.

# In Sage, I prefer testing in n=20, which is large enough to show differences
# but small enough to show 
n = 24 

# Set up the search, running Algorithm 2
search = Normal_Search(n, complexity_check, lex_first_check, normality_check, pre, update_conjugates)
# Use the %time line magic to time the search.
%time search.run()
# Use %prun line magic to give verbose profiling information about the program. SLOW.
# %prun search.run()


In [ ]:
## The remainder of these code blocks are used to verify that the
## output of our runs do indeed define normal elements and are 
## provide the stated complexities.

In [55]:
n = 40
farray = [7,9,13,14,15,16,18,20,22,23,24,30,31,33,35,36]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen()^3 + Fx.gen()^4 + Fx.gen()^5 + Fx.gen()^40
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y') 
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]

assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^40 + x^39 + x^37 + x^34 + x^31 + x^26 + x^24 + x^23 + x^21 + x^19 + x^18 + x^16 + x^9 + x^5 + 1
189


In [54]:
n = 41
farray = [0,5,6,8,9,11,13,14,15,17,18,19,22,24,25,27,28,29,31,34,35,37]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen()^3 + Fx.gen()^(41)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^41 + x^40 + x^38 + x^37 + x^36 + x^33 + x^32 + x^22 + x^21 + x^20 + x^17 + x^16 + x^9 + x^8 + x^6 + x^5 + x^4 + x + 1
81


In [57]:
n = 42
farray = [6,10,12,13,16,17,19,20,22,25,30,33,35,36,37,38,39,40]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen() + Fx.gen()^2 + Fx.gen()^5 + Fx.gen()^(42)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^42 + x^41 + x^40 + x^38 + x^36 + x^35 + x^31 + x^30 + x^26 + x^23 + x^22 + x^20 + x^19 + x^18 + x^15 + x^12 + x^3 + x^2 + 1
135


In [58]:
n = 43
farray = [6,7,9,10,12,18,20,21,22,24,26,28,31,32,33,34,37]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen()^3 + Fx.gen()^4 + Fx.gen()^6 + Fx.gen()^(43)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^43 + x^42 + x^40 + x^37 + x^35 + x^33 + x^31 + x^30 + x^29 + x^28 + x^27 + x^25 + x^24 + x^22 + x^20 + x^18 + x^14 + x^12 + x^11 + x^9 + x^8 + x^7 + x^5 + x^3 + 1
165


In [59]:
n = 44
farray = [6,9,11,13,16,17,21,22,24,29,31,33,34,35,39,40,42]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen()^5 + Fx.gen()^(44)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^44 + x^43 + x^42 + x^40 + x^35 + x^33 + x^30 + x^28 + x^27 + x^26 + x^25 + x^24 + x^23 + x^21 + x^19 + x^18 + x^17 + x^12 + x^11 + x^10 + x^9 + x^5 + x^3 + x^2 + 1
147


In [62]:
n = 45
farray = [9,12,13,14,17,19,21,24,26,27,29,31,34,36,37,38,40,41,44]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen() + Fx.gen()^3 + Fx.gen()^4 + Fx.gen()^(45)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

x^45 + x^44 + x^42 + x^41 + x^40 + x^39 + x^38 + x^37 + x^33 + x^30 + x^23 + x^20 + x^18 + x^16 + x^14 + x^13 + x^12 + x^11 + x^9 + x^6 + x^2 + x + 1
153


In [ ]:
n = 46
farray = [0,1,2,8,12,17,18,19,22,25,26,30,32,36,38,40,41,42,43,44,45]

F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen() + Fx.gen()^(46)
K = GF(2**n, name='x', modulus = modulus)
g = sum( [K.gen()**f for f in farray])
congs = [g**(2**(n-1-i)) for i in range(n) ]

y = var('y')
Ky = PolynomialRing(K,y)

h = Ky(0)
for i in range(n):
    h += Ky.gen()**i * congs[i]
    
assert Ky.gcd(h, Ky.gen()**n - 1) == 1  ###Normality test.

w = congs[0]
print w.minimal_polynomial()
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ])
P = matrix( [ congs[i]._vector_() for i in range(n) ])
Bpinv = B*P.inverse()

print len(Bpinv.nonzero_positions())

In [ ]:
## The following codeblock appears in the final submission and verifies that
## the provided minimal polynomial defines the a normal basis of the stated
## complexity

In [68]:
minpoly45 = w.minimal_polynomial()
print minpoly45
F = GF(2)
Fx = PolynomialRing(GF(2), 'x')
modulus = 1 + Fx.gen() + Fx.gen()^3 + Fx.gen()^4 + Fx.gen()^(45) #The NTL default modulus for n=45
K = GF(2**n, name='x', modulus=modulus) #GF(2^n) = GF(2)[x]/(modulus)
congs = minpoly45.roots(K, multiplicities=False) #The roots of minpoly45 are a set of conjugates
B = matrix([ (congs[i]*congs[0])._vector_() for i in range(n) ]) #Construct the matrix B_{cong[0]}
P = matrix( [ congs[i]._vector_() for i in range(n) ]) #Construct the matrix P_{\cong[0]}
Bpinv = B*P.inverse() #Construct BP^{-1} 

print len(Bpinv.nonzero_positions()) #The complexity of the basis is the number of nonzero positions in the matrix

x^45 + x^44 + x^42 + x^41 + x^40 + x^39 + x^38 + x^37 + x^33 + x^30 + x^23 + x^20 + x^18 + x^16 + x^14 + x^13 + x^12 + x^11 + x^9 + x^6 + x^2 + x + 1
153
